# Business Understanding

Linear Regression
* Fast
* Easily interpretable feature effect
* Easily evaluated
* Easily explained to non-technical
* Easily visualized

#### Assumptions of Linear Regression
* Linear relationship
* Normality
* Little to no multi-collinearity
* Evenly distributed(heteroskedastic) residuals.
* Non-autocorrelating residuals

# Data Understanding

* Data Types
    * Ints, Floats, Strings, Booleans
    * Numerical vs categorical?
* Missing data?
* Distribution
    * Range
    * Skew
* Correlation
* Visualization

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/kc_house_data.csv')

## Look at the data!

Bread and butter of first examinations: head(), info(), describe(),

In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
list(df.columns)

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

Taking a look at the column names, it is imperative to have an explanation of what the data in the column describes.  
For public dataset a quick search(google 'kc housing data metadata') will often find what is needed.  
If it is not public, ask up the chain where the data came from.  
On kaggle, there is a version of this dataset with a description of the columns: https://www.kaggle.com/harlfoxem/housesalesprediction.

Examine the 'info' about the data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


Things to take note of:
* 21 Total columns
* Total number of rows: 21597
    * Waterfront, view, and yr_renovated have less entries(nulls are present)
* Mix of integer(int64), floating point(float64) and string(object) data types present
* The 'date' and 'sqft_basement' columns are strings - these should be converted.

Examine the number of rows in each column that are missing data.  

In [6]:
df.isna().sum() 

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [82]:
nan_cols = [col for col in df.columns if df[col].isna().any()]
for col in nan_cols:
    total_nans = df[col].isna().sum()
    pct_nans = 100 * total_nans / len(df)
    print(f"{round(pct_nans, 2)} percent of {col} is missing")

11.0 percent of waterfront is missing
0.29 percent of view is missing
17.79 percent of yr_renovated is missing


__NOTE ON NON_NULL MISSING DATA__  
Technically there may still be missing data. Sometimes there is non-valid data entered as an empty string('') or irrelevant numbers(-999).  
More on this in a bit.

Use describe to take a quick look at distributions. (Can be a good way to spot '-999s')

In [7]:
df.describe().T     # Only describes numeric columns

,count,mean,std,min,25%,50%,75%,max
id,21597.0,4.580474e+09,2.876736e+09,1.000102e+06,2.123049e+09,3.904930e+09,7.308900e+09,9.900000e+09
price,21597.0,5.402966e+05,3.673681e+05,7.800000e+04,3.220000e+05,4.500000e+05,6.450000e+05,7.700000e+06
bedrooms,21597.0,3.373200e+00,9.262989e-01,1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.300000e+01
bathrooms,21597.0,2.115826e+00,7.689843e-01,5.000000e-01,1.750000e+00,2.250000e+00,2.500000e+00,8.000000e+00
sqft_living,21597.0,2.080322e+03,9.181061e+02,3.700000e+02,1.430000e+03,1.910000e+03,2.550000e+03,1.354000e+04
sqft_lot,21597.0,1.509941e+04,4.141264e+04,5.200000e+02,5.040000e+03,7.618000e+03,1.068500e+04,1.651359e+06
floors,21597.0,1.494096e+00,5.396828e-01,1.000000e+00,1.000000e+00,1.500000e+00,2.000000e+00,3.500000e+00
waterfront,19221.0,7.595859e-03,8.682485e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
view,21534.0,2.338627e-01,7.656862e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
condition,21597.0,3.409825e+00,6.505456e-01,1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00


Ultimately vizualizing the data with histograms will give us a more easy to read glimpse at distribution but this is a great first look.

What about not numeric data?

In [8]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront       float64
view             float64
condition          int64
grade              int64
sqft_above         int64
sqft_basement     object
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [9]:
df.select_dtypes('O')     # Viewing columns of a DataFrame where the values are strings - 'O' short for 'object'

,date,sqft_basement
0,10/13/2014,0.0
1,12/9/2014,400.0
2,2/25/2015,0.0
3,12/9/2014,910.0
4,2/18/2015,0.0
...,...,...
21592,5/21/2014,0.0
21593,2/23/2015,0.0
21594,6/23/2014,0.0
21595,1/16/2015,0.0


Both of these ultimately shouldn't be string. They should be datetime and float types. Can we change them quickly?

Converting to datetime:

In [43]:
df.date = pd.to_datetime(df.date)
df.dtypes[:4]

id                   int64
date        datetime64[ns]
price              float64
bedrooms             int64
dtype: object

Converting to float:

In [52]:
# df.sqft_basement = df.sqft_basement.astype('float')

The above will throw an error for the '?' value in sqft_basement.  
This is an example of the NON_NULL missing data mentioned above.  
We will want to remove or fill these cells, depending on how many there are.

In [56]:
df.sqft_basement.value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
176.0         1
2120.0        1
1798.0        1
768.0         1
2190.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [61]:
sum(df.sqft_basement == '?')/len(df)

0.021021438162707785

2.1% of the __sqft_basement__ is missing.
And from before  
11.0% of __waterfront__ is missing  
0.29% of __view__ is missing  
17.79% of __yr_renovated__ is missing

## Missing Data
Missing data needs to be filled or dropped. If few rows are missing, dropping is likely the best step. If many rows are missing and the column is contains impactful data, then the rows should be filled. Nan's are most commonly filled with the mean or the median depending on the distribution.

Another option is to bin the data and create categorical features based on the bins and leave the nans in a bin of their own.

__sqft_basement__: ...
__waterfront__: Dropping 11% is a significant loss of information and is rarely the best option. It should be filled.  
__view__: Dropping 0.3% of data will not lose a lot of information and therefore dropping is a viable option for __view__
__yr_renovated__: Dropping ...

* Data Types
    * Ints, Floats, Strings, Booleans
    * Numerical vs categorical?
* Missing data?
* Distribution
    * Range
    * Skew
* Correlation
* Visualization

# Data Preparation

* Coerce any dtypes to proper format
* Normalize numerical
* Remove, or fill missing data
* One Hot Encode Categorical Data

# Modeling

# Evaluation

# Deployment

Putting in a script.py and running it